# NIRCam PSF 光度测量与 Space_Phot

**作者**: Ori Fox

<br>

**最后更新**: 2025年1月

## 目录

1. [介绍](#intro)<br>

2. [设置](#setup)<br>

    2.1 [Python 导入](#py_imports)<br>

    2.2 [下载数据](#bso4)<br>

3. [明亮的单一对象](#bso)<br>

    3.1 [多个 Level2 文件](#bso2)<br>

4. [微弱/上限，单一对象](#fso)<br>

    4.1 [多个 Level2 文件](#fso2)<br>

5. [恒星场 (LMC)](#lmv)<br>

    5.1 [多个 Level2 文件](#lmc2)<br>

    5.2 [单个 Level3 马赛克文件](#lmc3)<br>

1.<font color='white'>-</font>介绍 <a class="anchor" id="intro"></a>

------------------

**要安装的包**：

drizzlepac\\

space_phot https://github.com/jpierel14/space_phot\\

photutils（在主分支 git+https://github.com/astropy/photutils）\\

jupyter\\

**目标**： 

PSF（点扩散函数）光度测量可以通过以下方式获得：

* 来自WebbPSF的PSF模型网格

* 单一有效PSF（ePSF）尚不可用

* 有效PSF网格尚不可用

该笔记本展示了：

* 如何从WebbPSF获取PSF模型（或构建ePSF）

* 如何在图像上执行PS

* 光度测量

2.<font color='white'>-</font>设置 <a class="anchor" id="setup"></a>

------------------

### 2.1<font color='white'>-</font>Python 导入<a class="anchor" id="py_imports"></a> ###

In [ ]:
from astropy.io import fits  # 导入FITS文件处理模块

from astropy.nddata import extract_array  # 导入数组提取功能

from astropy.coordinates import SkyCoord  # 导入天文坐标处理模块

from astropy import wcs  # 导入世界坐标系统模块

from astropy.wcs.utils import skycoord_to_pixel  # 导入天文坐标到像素坐标的转换功能

from astropy import units as u  # 导入单位模块

import numpy as np  # 导入NumPy库

import pandas as pd  # 导入Pandas库

from astropy.visualization import simple_norm  # 导入简单归一化功能

from urllib.parse import urlparse  # 导入URL解析模块

import requests  # 导入请求库

import time  # 导入时间模块

import math  # 导入数学模块

import logging  # 导入日志模块

from jwst.associations import load_asn  # 导入JWST关联加载功能

import matplotlib.pyplot as plt  # 导入Matplotlib绘图库

%matplotlib inline  # 在Jupyter Notebook中内联显示Matplotlib图形

from astroquery.mast import Observations  # 导入MAST观测数据查询模块

import os  # 导入操作系统模块

import tarfile  # 导入tar文件处理模块

# 背景和点扩散函数（PSF）相关函数

from photutils.background import MMMBackground, MADStdBackgroundRMS  # 导入背景估计功能

from photutils.detection import DAOStarFinder  # 导入星点检测功能

import space_phot  # 导入空间光度测量模块

from importlib.metadata import version  # 导入获取模块版本的功能

version('space_phot')  # 获取并显示'space_phot'模块的版本

### 2.2<font color='white'>-</font>下载数据<a class="anchor" id="bso4"></a> ###

In [ ]:
def get_asn_filenames(program, observation, objnum, filtername):
    """查询MAST以确定给定程序、观测和对象编号的关联文件名。
    
    这个函数比硬编码文件名更方便，因为文件名包含了生成日期戳。
    因此，每次在MAST中重新处理文件时，文件名都会发生变化。

    参数
    ----------
    program : int
        程序ID号，例如1067

    observation : int
        观测编号，例如24

    objnum : int
        对象编号。

    filtername : str
        观测中使用的滤光片名称，例如"F444W"

    返回
    -------
    files_to_download : list
        匹配输入参数的文件名列表
    """
    
    prog_str = str(program).zfill(5)  # 将程序ID转换为5位字符串，不足补零
    obs_str = str(observation).zfill(3)  # 将观测编号转换为3位字符串，不足补零
    obj_str = str(objnum).zfill(5)  # 将对象编号转换为5位字符串，不足补零

    # 查询符合条件的观测数据
    obs_id_table = Observations.query_criteria(instrument_name=["NIRCAM/IMAGE"],
                                               provenance_name=["CALJWST"],  # 执行的观测
                                               obs_id=['jw' + prog_str + '-o' + obs_str + '*'],  # 生成观测ID
                                               filters=[filtername.upper()]  # 使用指定的滤光片
                                               )

    matching_files = []  # 初始化匹配文件列表

    for exposure in (obs_id_table):  # 遍历查询到的观测数据
        products = Observations.get_product_list(exposure)  # 获取产品列表
        filtered_products = Observations.filter_products(products, productSubGroupDescription='ASN')  # 过滤出ASN产品
        matching_files.extend(filtered_products['dataURI'])  # 将匹配的文件URI添加到列表中

    # 过滤出包含特定对象编号的文件名
    matching_files = [os.path.basename(e) for e in matching_files if f'_image3_{obj_str}_asn' in e]

    return matching_files  # 返回匹配的文件名列表

In [ ]:
def download_files(files_to_download):
    """从MAST下载文件列表。

    Parameters
    ----------
    files_to_download : list
        文件名列表
    """

    for file in files_to_download:
        # 检查文件是否已存在于当前工作目录中
        if os.path.exists(file):
            print(f"文件 {file} 已存在。跳过下载。")
            continue

        # 构建文件的URI
        cal_uri = f'mast:JWST/product/{file}'

        # 下载文件
        Observations.download_file(cal_uri)

In [ ]:
# 下载NIRCam数据 PID 1537（校准程序）和NIRCam数据 PID 1476（LMC）

files_to_download = ['jw01537-o024_t001_nircam_clear-f444w-sub160_i2d.fits',  # NIRCam F444W 数据文件
                     'jw01537024001_0310a_00001_nrcblong_cal.fits',  # NIRCam 长曝光校准文件 1
                     'jw01537024001_0310a_00002_nrcblong_cal.fits',  # NIRCam 长曝光校准文件 2
                     'jw01537024001_0310a_00003_nrcblong_cal.fits',  # NIRCam 长曝光校准文件 3
                     'jw01537024001_0310a_00004_nrcblong_cal.fits',  # NIRCam 长曝光校准文件 4
                     'jw01537024001_0310k_00001_nrcblong_cal.fits',  # NIRCam 长曝光校准文件 5
                     'jw01537024001_0310k_00002_nrcblong_cal.fits',  # NIRCam 长曝光校准文件 6
                     'jw01537024001_0310k_00003_nrcblong_cal.fits',  # NIRCam 长曝光校准文件 7
                     'jw01537024001_0310k_00004_nrcblong_cal.fits',  # NIRCam 长曝光校准文件 8
                     'jw01476-o001_t001_nircam_clear-f150w_i2d.fits',  # NIRCam F150W 数据文件
                     'jw01476001007_02101_00001_nrca1_cal.fits',  # NIRCam 校准文件 1
                     'jw01476001007_02101_00002_nrca1_cal.fits',  # NIRCam 校准文件 2
                     'jw01476001007_02101_00003_nrca1_cal.fits',  # NIRCam 校准文件 3
                     'jw01476001008_02101_00001_nrca1_cal.fits',  # NIRCam 校准文件 4
                     'jw01476001008_02101_00002_nrca1_cal.fits',  # NIRCam 校准文件 5
                     'jw01476001008_02101_00003_nrca1_cal.fits',  # NIRCam 校准文件 6
                     'jw01476001008_02101_00004_nrca1_cal.fits',  # NIRCam 校准文件 7
                     'jw01476001008_02101_00005_nrca1_cal.fits',  # NIRCam 校准文件 8
                     'jw01476001008_02101_00006_nrca1_cal.fits'   # NIRCam 校准文件 9
                     ]

# 获取相关关联文件的名称并将其添加到下载文件列表中
asn_files_to_download = [get_asn_filenames(1537, 24, 1, 'F444W')[0],  # 获取PID 1537的关联文件
                         get_asn_filenames(1476, 1, 23, 'F150W')[0]   # 获取PID 1476的关联文件
                         ]

files_to_download += asn_files_to_download  # 将关联文件添加到下载列表中

# 调用函数下载文件
download_files(files_to_download)  # 下载所有指定的文件

3.<font color='white'>-</font>明亮的单一天体<a class="anchor" id="bso"></a>

------------------

### 3.1<font color='white'>-</font>多个 Level2 文件<a class="anchor" id="bso2"></a> ###

In [ ]:
# Level 3 Files: NIRCam Data PID 1537 (Calibration Program):
# 定义Level 3文件名，包含NIRCam数据的PID和相关信息
lvl3 = 'jw01537-o024_t001_nircam_clear-f444w-sub160_i2d.fits'

# 输出文件名
lvl3

In [ ]:
hdl = fits.open(lvl3)  # 打开指定的FITS文件，返回一个HDU列表

hdr = hdl[0].header  # 获取第一个HDU的头部信息

asnfile = hdr['ASNTABLE']  # 从头部信息中提取ASNTABLE字段的值，获取关联文件名

lvl2_prelim = []  # 初始化一个空列表，用于存储二级数据的文件名

asn_data = load_asn(open(asnfile))  # 加载关联文件，解析其内容

for member in asn_data['products'][0]['members']:  # 遍历第一个产品的所有成员

    #print(member['expname'])  # 打印每个成员的曝光名称（可选）

    lvl2_prelim.append(member['expname'])  # 将每个成员的曝光名称添加到列表中

lvl2_prelim  # 返回包含所有二级数据文件名的列表

In [ ]:
# 整理包含实际源的LVL2数据（共有4个探测器）

source_location = SkyCoord('5:05:30.6593', '+52:49:49.862', unit=(u.hourangle, u.deg))  # 定义源位置

lvl2 = []  # 初始化LVL2数据列表

for ref_image in lvl2_prelim:  # 遍历预处理的LVL2图像

    print(ref_image)  # 打印当前图像名称

    ref_fits = fits.open(ref_image)  # 打开FITS文件

    ref_data = fits.open(ref_image)['SCI', 1].data  # 获取科学数据部分

    ref_y, ref_x = skycoord_to_pixel(source_location, wcs.WCS(ref_fits['SCI', 1], ref_fits))  # 将天球坐标转换为像素坐标

    print(ref_y, ref_x)  # 打印转换后的像素坐标

    try:
        extract_array(ref_data, (11, 11), (ref_x, ref_y))  # 提取以源位置为中心的11x11像素块，可能会引发异常

    except Exception as e:  # 捕获异常
        logging.error(f"An error occurred: {e}")  # 记录错误信息
        pass  # 在异常发生时不做任何操作，但记录日志

    else:  # 如果没有异常发生
        lvl2.append(ref_image)  # 将当前图像添加到LVL2列表中
        print(ref_image + ' added to final list')  # 打印添加成功的信息

lvl2  # 返回最终的LVL2数据列表

In [ ]:
# 将所有DQ标记的像素值更改为NAN

for file in lvl2:  # 遍历所有的lvl2文件

    hdul = fits.open(file, mode='update')  # 以更新模式打开FITS文件

    data = fits.open(file)['SCI', 1].data  # 读取科学数据（SCI）部分

    dq = fits.open(file)['DQ', 1].data  # 读取数据质量（DQ）部分

    data[dq == 1] = np.nan  # 将DQ标记为1的像素值设置为NAN

    hdul['SCI', 1].data = data  # 更新科学数据部分

    hdul.flush()  # 刷新文件以保存更改

In [ ]:
# 检查第一幅图像

ref_image = lvl2[0]  # 获取第二级数据中的第一幅图像

print(ref_image)  # 打印图像的文件名或路径

ref_fits = fits.open(ref_image)  # 打开FITS文件以读取数据

ref_data = fits.open(ref_image)['SCI', 1].data  # 从FITS文件中提取科学数据部分

norm1 = simple_norm(ref_data, stretch='linear', min_cut=-1, max_cut=10)  # 规范化数据以便于可视化

plt.imshow(ref_data, origin='lower', norm=norm1, cmap='gray')  # 显示图像，使用灰度色图

plt.gca().tick_params(labelcolor='none', axis='both', color='none')  # 隐藏坐标轴的刻度和标签

plt.show()  # 显示图像

lvl2[0]  # 返回第二级数据中的第一幅图像（此行似乎没有实际作用）

In [ ]:
# 放大以查看源
ref_y, ref_x = skycoord_to_pixel(source_location, wcs.WCS(ref_fits['SCI', 1], ref_fits))  # 将天球坐标转换为像素坐标

ref_cutout = extract_array(ref_data, (11, 11), (ref_x, ref_y))  # 从数据中提取一个11x11的切片

norm1 = simple_norm(ref_cutout, stretch='linear', min_cut=-10, max_cut=1000)  # 创建归一化对象，使用线性拉伸

plt.imshow(ref_cutout, origin='lower',  # 显示切片图像，设置原点在下方
           norm=norm1, cmap='gray')  # 使用灰度色图和归一化对象

plt.title('PID1537,Obs024')  # 设置图像标题

plt.gca().tick_params(labelcolor='none', axis='both', color='none')  # 隐藏坐标轴刻度

plt.show()  # 显示图像

ref_cutout  # 返回切片数据

In [ ]:
import os  # 导入操作系统模块
import requests  # 导入请求模块
import tarfile  # 导入tarfile模块，用于处理tar文件
from urllib.parse import urlparse  # 导入url解析模块
import matplotlib.pyplot as plt  # 导入绘图库

# 设置环境变量
os.environ["WEBBPSF_PATH"] = "./webbpsf-data/webbpsf-data"  # 设置WebbPSF数据路径
os.environ["PYSYN_CDBS"] = "./grp/redcat/trds/"  # 设置Pysyn数据库路径

# 所需的WebbPSF数据
boxlink = 'https://stsci.box.com/shared/static/qxpiaxsjwo15ml6m4pkhtk36c9jgj70k.gz'  # 数据压缩包链接
boxfile = './webbpsf-data/webbpsf-data-LATEST.tar.gz'  # 本地保存的压缩包路径
synphot_url = 'http://ssb.stsci.edu/trds/tarfiles/synphot5.tar.gz'  # Synphot数据链接
synphot_file = './synphot5.tar.gz'  # 本地保存的Synphot压缩包路径

webbpsf_folder = './webbpsf-data'  # WebbPSF数据文件夹路径
synphot_folder = './grp'  # Synphot数据文件夹路径

# 下载文件的函数
def download_file(url, dest_path, timeout=60):
    parsed_url = urlparse(url)  # 解析URL

    if parsed_url.scheme not in ["http", "https"]:  # 检查URL协议
        raise ValueError(f"Unsupported URL scheme: {parsed_url.scheme}")  # 抛出不支持的协议错误

    response = requests.get(url, stream=True, timeout=timeout)  # 发送GET请求
    response.raise_for_status()  # 检查请求是否成功
    with open(dest_path, "wb") as f:  # 以二进制写入模式打开文件
        for chunk in response.iter_content(chunk_size=8192):  # 分块读取内容
            f.write(chunk)  # 写入文件

# 收集WebbPSF文件
psfExist = os.path.exists(webbpsf_folder)  # 检查WebbPSF文件夹是否存在

if not psfExist:  # 如果文件夹不存在
    os.makedirs(webbpsf_folder)  # 创建文件夹
    download_file(boxlink, boxfile)  # 下载WebbPSF数据
    gzf = tarfile.open(boxfile)  # 打开压缩包
    gzf.extractall(webbpsf_folder)  # 解压缩到指定文件夹

# 收集Synphot文件
synExist = os.path.exists(synphot_folder)  # 检查Synphot文件夹是否存在

if not synExist:  # 如果文件夹不存在
    os.makedirs(synphot_folder)  # 创建文件夹
    download_file(synphot_url, synphot_file)  # 下载Synphot数据
    gzf = tarfile.open(synphot_file)  # 打开压缩包
    gzf.extractall('./')  # 解压缩到当前目录

# 从WebbPSF获取点扩散函数(PSF)
jwst_obs = space_phot.observation2(lvl2)  # 获取JWST观测数据
psfs = space_phot.get_jwst_psf(jwst_obs, source_location)  # 获取JWST的PSF
plt.imshow(psfs[0].data)  # 显示PSF数据
plt.show()  # 展示图像

In [ ]:
# 使用 space_phot 进行 PSF 光度测量（拟合的详细信息请参见文档）

# https://st-phot.readthedocs.io/en/latest/examples/plot_a_psf.html#jwst-images

# 调用 JWST 观测对象的 PSF 光度测量方法
jwst_obs.psf_photometry(
    psfs,  # 输入的点扩散函数（PSF）
    source_location,  # 源的位置
    bounds={  # 设置参数的边界
        'flux': [-10, 10000],  # 流量的边界
        'centroid': [-2, 2],  # 中心位置的边界
        'bkg': [0, 50]  # 背景的边界
    },
    fit_width=5,  # 拟合宽度
    fit_bkg=True,  # 是否拟合背景
    fit_flux='single'  # 流量拟合模式
)

# 绘制 PSF 拟合结果
jwst_obs.plot_psf_fit()
plt.show()  # 显示图像

# 绘制 PSF 后验分布
jwst_obs.plot_psf_posterior(minweight=.0005)  # 设置最小权重
plt.show()  # 显示图像

# 打印 PSF 结果的光度校准表
print(jwst_obs.psf_result.phot_cal_table)  # 输出光度校准表

In [ ]:
# 从表格中计算平均星等

mag_arr = jwst_obs.psf_result.phot_cal_table['mag']  # 获取星等数组

magerr_arr = jwst_obs.psf_result.phot_cal_table['magerr']  # 获取星等误差数组

mag_lvl2psf = np.mean(mag_arr)  # 计算星等的平均值

magerr_lvl2psf = math.sqrt(sum(p**2 for p in magerr_arr))  # 计算误差的平方和的平方根

print(round(mag_lvl2psf, 4), round(magerr_lvl2psf, 4))  # 输出平均星等和误差，保留四位小数

4.<font color='white'>-</font>微弱/上限，单个天体<a class="anchor" id="fso"></a>

------------------

### 4.1<font color='white'>-</font>多个 Level2 文件<a class="anchor" id="fso2"></a> ###

In [ ]:
# Level 3 Files

# 定义JWST的Level 3文件名
lvl3 = 'jw01537-o024_t001_nircam_clear-f444w-sub160_i2d.fits'

# 输出文件名
lvl3

In [ ]:
from jwst.associations import load_asn  # 从JWST关联模块导入load_asn函数

hdl = fits.open(lvl3)  # 打开给定的lvl3文件

hdr = hdl[0].header  # 获取文件的头部信息

asnfile = hdr['ASNTABLE']  # 从头部信息中提取关联表文件名

lvl2_prelim = []  # 初始化一个空列表，用于存储lvl2初步数据

asn_data = load_asn(open(asnfile))  # 加载关联数据

for member in asn_data['products'][0]['members']:  # 遍历第一个产品的所有成员

    lvl2_prelim.append(member['expname'])  # 将每个成员的曝光名称添加到lvl2_prelim列表中

lvl2_prelim  # 返回lvl2_prelim列表

In [ ]:
# 整理包含实际源的LVL2数据（共有4个探测器）

source_location = SkyCoord('5:05:30.6186', '+52:49:49.130', unit=(u.hourangle, u.deg))  # 定义源位置的天球坐标

lvl2 = []  # 初始化LVL2数据列表

for ref_image in lvl2_prelim:  # 遍历初步LVL2图像列表

    print(ref_image)  # 打印当前处理的图像名称

    ref_fits = fits.open(ref_image)  # 打开FITS文件以读取数据

    ref_data = fits.open(ref_image)['SCI', 1].data  # 获取科学数据部分

    ref_y, ref_x = skycoord_to_pixel(source_location, wcs.WCS(ref_fits['SCI', 1], ref_fits))  # 将天球坐标转换为像素坐标

    print(ref_y, ref_x)  # 打印转换后的像素坐标

    try:
        extract_array(ref_data, (11, 11), (ref_x, ref_y))  # 提取以源位置为中心的11x11像素块，可能会引发异常

    except Exception as e:  # 捕获异常
        logging.error(f"An error occurred: {e}")  # 记录错误信息
        pass  # 在异常发生时不做任何处理，但记录日志

    else:  # 如果没有异常发生
        lvl2.append(ref_image)  # 将当前图像添加到最终列表中
        print(ref_image + ' added to final list')  # 打印已添加图像的信息

lvl2  # 返回最终的LVL2数据列表

In [ ]:
# 将所有DQ标记的像素值更改为NAN

for file in lvl2:  # 遍历每个文件

    hdul = fits.open(file, mode='update')  # 以更新模式打开FITS文件

    data = fits.open(file)['SCI', 1].data  # 读取科学数据部分

    dq = fits.open(file)['DQ', 1].data  # 读取数据质量（DQ）部分

    data[dq == 1] = np.nan  # 将DQ标记为1的像素值更改为NAN

    hdul['SCI', 1].data = data  # 更新科学数据部分

    hdul.flush()  # 刷新更改到文件

In [ ]:
# 检查第一幅图像

ref_image = lvl2[0]  # 获取第二级数据中的第一幅图像

print(ref_image)  # 打印图像文件名

ref_fits = fits.open(ref_image)  # 打开FITS文件以读取数据

ref_data = fits.open(ref_image)['SCI', 1].data  # 从FITS文件中提取科学数据部分

norm1 = simple_norm(ref_data, stretch='linear', min_cut=-1, max_cut=25)  # 规范化数据以便于显示

plt.imshow(ref_data, origin='lower', norm=norm1, cmap='gray')  # 显示图像，设置原点在下方，使用灰度色图

plt.gca().tick_params(labelcolor='none', axis='both', color='none')  # 隐藏坐标轴刻度和标签

plt.show()  # 显示图像

In [ ]:
# 选择天空中一个空白区域以计算上限

# 将源位置转换为像素坐标
ref_y, ref_x = skycoord_to_pixel(source_location, wcs.WCS(ref_fits['SCI', 1], ref_fits))

# 从数据中提取一个11x11的切片
ref_cutout = extract_array(ref_data, (11, 11), (ref_x, ref_y))

# 对切片进行归一化处理，使用线性拉伸，设置最小和最大裁剪值
norm1 = simple_norm(ref_cutout, stretch='linear', min_cut=-1, max_cut=25)

# 显示切片图像，设置原点在下方，使用灰度色图
plt.imshow(ref_cutout, origin='lower',
           norm=norm1, cmap='gray')

# 设置图像标题
plt.title('PID1028,Obs006')

# 隐藏坐标轴的刻度标签和颜色
plt.gca().tick_params(labelcolor='none', axis='both', color='none')

# 显示图像
plt.show()

In [ ]:
# 从WebbPSF获取点扩散函数（PSF）

jwst_obs = space_phot.observation2(lvl2)  # 将lvl2数据转换为JWST观测对象

psfs = space_phot.get_jwst_psf(jwst_obs, source_location)  # 获取指定源位置的JWST PSF

plt.imshow(psfs[0].data)  # 显示第一个PSF的图像数据

plt.show()  # 展示图像

In [ ]:
# 使用 space_phot 进行 PSF 光度测量（详细的拟合信息请参见文档）

# https://st-phot.readthedocs.io/en/latest/examples/plot_a_psf.html#jwst-images

# 调用 jwst_obs 的 psf_photometry 方法进行 PSF 光度测量
jwst_obs.psf_photometry(

    psfs,  # 输入的 PSF 数据

    source_location,  # 源位置

    bounds={  # 设置参数的边界
        'flux': [-10, 1000],  # 流量的边界
        'bkg': [0, 50]  # 背景的边界
    },

    fit_width=5,  # 拟合宽度设置为 5 像素

    fit_bkg=True,  # 是否拟合背景
    fit_centroid='fixed',  # 中心位置固定
    fit_flux='single'  # 拟合单一流量
)

# 绘制 PSF 拟合结果
jwst_obs.plot_psf_fit()
plt.show()  # 显示图像

# 绘制 PSF 后验分布
jwst_obs.plot_psf_posterior(minweight=.0005)
plt.show()  # 显示图像

# 打印 PSF 结果的光度校准表
print(jwst_obs.psf_result.phot_cal_table)

In [ ]:
# 打印上限

# 计算JWST观测的上限，使用5个标准差
magupper_lvl2psf = jwst_obs.upper_limit(nsigma=5)

# 输出计算得到的上限值
magupper_lvl2psf

5.<font color='white'>-</font>恒星场 (LMC)<a class="anchor" id="lmc"></a>

------------------

### 5.1<font color='white'>-</font>多个 Level2 文件<a class="anchor" id="lmc2"></a> ###

##### 现在对一组更大的恒星进行相同的操作，并测试速度

In [ ]:
# Level 3 Files: NIRCam Data PID 1476 (LMC)

# 定义文件名，包含JWST NIRCam数据的Level 3文件
lvl3 = 'jw01476-o001_t001_nircam_clear-f150w_i2d.fits'

# 输出文件名
lvl3

In [ ]:
hdl = fits.open(lvl3)  # 打开给定的FITS文件，返回一个HDUList对象

hdr = hdl[0].header  # 获取FITS文件的头部信息

asnfile = hdr['ASNTABLE']  # 从头部信息中提取关联文件名

lvl2 = []  # 初始化一个空列表，用于存储符合条件的文件名

asn_data = load_asn(open(asnfile))  # 加载关联文件的数据

for member in asn_data['products'][0]['members']:  # 遍历关联文件中的每个成员
    lvl2.append(member['expname'])  # 将每个成员的文件名添加到lvl2列表中

lvl2 = [s for s in lvl2 if "nrca1" in s]  # 过滤出文件名中包含"nrca1"的文件

lvl2  # 输出最终的lvl2列表

In [ ]:
# 在Level 3文件中查找恒星

# 获取背景的粗略估计（有更好的方法进行背景减法）
bkgrms = MADStdBackgroundRMS()  # 创建MAD标准背景RMS对象
mmm_bkg = MMMBackground()        # 创建MMM背景对象

im = fits.open(lvl3)             # 打开Level 3 FITS文件

w = wcs.WCS(im['SCI', 1])        # 获取WCS信息

std = bkgrms(im[1].data)         # 计算背景的标准差
bkg = mmm_bkg(im[1].data)        # 计算背景值
data_bkgsub = im[1].data.copy()  # 复制数据以进行背景减法
data_bkgsub -= bkg                # 从数据中减去背景值

sigma_psf = 1.636                 # F770W的PSF标准差（以像素为单位）
threshold = 5.                    # 设置阈值

daofind = DAOStarFinder(threshold=threshold * std, fwhm=sigma_psf, exclude_border=True)  # 创建DAO星体查找器
found_stars = daofind(data_bkgsub)  # 在背景减法后的数据中查找恒星

In [ ]:
found_stars.pprint_all(max_lines=10)  # 打印找到的星星信息，最多显示10行

In [ ]:
# 过滤出你想要的恒星

plt.figure(figsize=(12, 8))  # 创建一个12x8英寸的图形

plt.clf()  # 清空当前图形

ax1 = plt.subplot(2, 1, 1)  # 创建一个2行1列的子图，选择第一个子图

ax1.set_xlabel('mag')  # 设置x轴标签为'mag'（星等）

ax1.set_ylabel('sharpness')  # 设置y轴标签为'sharpness'（锐度）

xlim0 = np.min(found_stars['mag']) - 0.25  # 计算x轴下限

xlim1 = np.max(found_stars['mag']) + 0.25  # 计算x轴上限

ylim0 = np.min(found_stars['sharpness']) - 0.15  # 计算y轴下限

ylim1 = np.max(found_stars['sharpness']) + 0.15  # 计算y轴上限

ax1.set_xlim(xlim0, xlim1)  # 设置x轴范围

ax1.set_ylim(ylim0, ylim1)  # 设置y轴范围

ax1.scatter(found_stars['mag'], found_stars['sharpness'], s=10, color='k')  # 绘制散点图

sh_inf = 0.40  # 锐度下限

sh_sup = 0.82  # 锐度上限

lmag_lim = -1.0  # 星等下限

umag_lim = -6.0  # 星等上限

ax1.plot([xlim0, xlim1], [sh_sup, sh_sup], color='r', lw=3, ls='--')  # 绘制锐度上限线

ax1.plot([xlim0, xlim1], [sh_inf, sh_inf], color='r', lw=3, ls='--')  # 绘制锐度下限线

ax1.plot([lmag_lim, lmag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')  # 绘制星等下限线

ax1.plot([umag_lim, umag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')  # 绘制星等上限线

ax2 = plt.subplot(2, 1, 2)  # 创建一个2行1列的子图，选择第二个子图

ax2.set_xlabel('mag')  # 设置x轴标签为'mag'（星等）

ax2.set_ylabel('roundness')  # 设置y轴标签为'roundness'（圆度）

ylim0 = np.min(found_stars['roundness2']) - 0.25  # 计算y轴下限

ylim1 = np.max(found_stars['roundness2']) - 0.25  # 计算y轴上限

ax2.set_xlim(xlim0, xlim1)  # 设置x轴范围

ax2.set_ylim(ylim0, ylim1)  # 设置y轴范围

round_inf = -0.40  # 圆度下限

round_sup = 0.40  # 圆度上限

ax2.scatter(found_stars['mag'], found_stars['roundness2'], s=10, color='k')  # 绘制散点图

ax2.plot([xlim0, xlim1], [round_sup, round_sup], color='r', lw=3, ls='--')  # 绘制圆度上限线

ax2.plot([xlim0, xlim1], [round_inf, round_inf], color='r', lw=3, ls='--')  # 绘制圆度下限线

ax2.plot([lmag_lim, lmag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')  # 绘制星等下限线

ax2.plot([umag_lim, umag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')  # 绘制星等上限线

plt.tight_layout()  # 调整子图参数，使之填充整个图像区域

In [ ]:
# 创建一个布尔掩码，筛选符合条件的恒星
mask = ((found_stars['mag'] < lmag_lim) &  # 过滤亮度小于下限
        (found_stars['mag'] > umag_lim) &  # 过滤亮度大于上限
        (found_stars['roundness2'] > round_inf) &  # 过滤圆度大于下限
        (found_stars['roundness2'] < round_sup) &  # 过滤圆度小于上限
        (found_stars['sharpness'] > sh_inf) &  # 过滤锐度大于下限
        (found_stars['sharpness'] < sh_sup) &  # 过滤锐度小于上限
        (found_stars['xcentroid'] > 1940) &  # 过滤x坐标大于1940
        (found_stars['xcentroid'] < 2000) &  # 过滤x坐标小于2000
        (found_stars['ycentroid'] > 1890) &  # 过滤y坐标大于1890
        (found_stars['ycentroid'] < 1960))  # 过滤y坐标小于1960

# 根据掩码选择符合条件的恒星
found_stars_sel = found_stars[mask]

# 打印原始找到的恒星数量
print('Number of stars found originally:', len(found_stars))

# 打印最终选择的恒星数量
print('Number of stars in final selection:', len(found_stars_sel))

In [ ]:
看起来您提到的 `found_stars_sel` 可能是一个变量或函数的名称，但没有提供具体的代码。如果您能提供相关的代码片段，我将能够为您添加中文注释并保持代码结构不变。请将代码粘贴在这里，我会尽快帮助您！

In [ ]:
# 将像素坐标转换为世界坐标系（WCS）坐标

# 使用像素坐标（xcentroid, ycentroid）转换为天球坐标
skycoords = w.pixel_to_world(found_stars_sel['xcentroid'], found_stars_sel['ycentroid'])

# 获取转换后的天球坐标的数量
len(skycoords)

In [ ]:
当然可以！不过您没有提供具体的代码。请您提供需要添加注释的JWST望远镜数据处理的Python代码，我将为您添加行级中文注释。

In [ ]:
file = lvl2[0]  # 从lvl2列表中获取第一个文件

dq = fits.open(file)['DQ', 1].data  # 打开文件并获取DQ扩展的第1个数据

dq[233, 340]  # 访问DQ数据数组中第234行、第341列的值

In [ ]:
# 将所有DQ标记的像素值更改为NAN

for file in lvl2:  # 遍历每个文件

    hdul = fits.open(file, mode='update')  # 以更新模式打开FITS文件

    data = fits.open(file)['SCI', 1].data  # 读取科学数据部分

    dq = fits.open(file)['DQ', 1].data  # 读取DQ数据部分

    data[dq == 262657] = np.nan  # 将DQ标记为262657的像素值设置为NAN

    data[dq == 262661] = np.nan  # 将DQ标记为262661的像素值设置为NAN

    hdul['SCI', 1].data = data  # 更新科学数据部分

    hdul.flush()  # 刷新文件以保存更改

In [ ]:
# 创建一个用于快速查找的网格，使用WebbPSF。网格越大，光度精度越高。

# 开发者注释：希望能够有一个快速/近似的查找表。

jwst_obs = space_phot.observation2(lvl2)  # 将lvl2数据转换为JWST观测对象

grid = space_phot.util.get_jwst_psf_grid(jwst_obs, num_psfs=4)  # 获取JWST PSF网格，指定PSF数量为4

In [ ]:
# 现在循环遍历所有恒星并构建光度表

counter = 0.  # 初始化计数器

badindex = []  # 初始化坏索引列表

# 从JWST观察数据中获取观测对象
jwst_obs = space_phot.observation2(lvl2)

# 遍历每个源位置
for source_location in skycoords:

    tic = time.perf_counter()  # 记录开始时间

    print('Starting', counter + 1., ' of', len(skycoords), ':', source_location)  # 打印当前处理的源位置

    # 从网格中获取JWST点扩散函数（PSF）
    psfs = space_phot.util.get_jwst_psf_from_grid(jwst_obs, source_location, grid)

    # 进行PSF光度测量
    jwst_obs.psf_photometry(
        psfs,
        source_location,
        bounds={
            'flux': [-100, 1000],  # 流量的边界
            'centroid': [-2., 2.],  # 重心的边界
            'bkg': [0, 50]  # 背景的边界
        },
        fit_width=3,  # 拟合宽度
        fit_bkg=False,  # 是否拟合背景
        fit_flux='single',  # 流量拟合类型
        maxiter=5000  # 最大迭代次数
    )

    jwst_obs.plot_psf_fit()  # 绘制PSF拟合图
    plt.show()  # 显示图形

    # 获取光度测量结果
    ra = jwst_obs.psf_result.phot_cal_table['ra'][0]  # 获取右升角
    dec = jwst_obs.psf_result.phot_cal_table['dec'][0]  # 获取 declination
    mag_arr = jwst_obs.psf_result.phot_cal_table['mag']  # 获取星等数组
    magerr_arr = jwst_obs.psf_result.phot_cal_table['magerr']  # 获取星等误差数组

    mag_lvl2psf = np.mean(mag_arr)  # 计算平均星等
    magerr_lvl2psf = math.sqrt(sum(p**2 for p in magerr_arr))  # 计算星等误差的平方和的平方根

    # 如果是第一次循环，初始化数据框
    if counter == 0:
        df = pd.DataFrame(np.array([[ra, dec, mag_lvl2psf, magerr_lvl2psf]]), columns=['ra', 'dec', 'mag', 'magerr'])
    else:
        # 否则，追加数据到数据框
        df = pd.concat([df, pd.DataFrame(np.array([[ra, dec, mag_lvl2psf, magerr_lvl2psf]]))], ignore_index=True)

    counter = counter + 1.  # 更新计数器

    toc = time.perf_counter()  # 记录结束时间
    print("Elapsed Time for Photometry:", toc - tic)  # 打印光度测量的耗时

### 5.2<font color='white'>-</font>单个 Level3 马赛克文件<a class="anchor" id="lmc3"></a> ###

In [ ]:
看起来您提到的“lvl3”可能是指某种数据处理或分析的级别，但没有提供具体的代码或上下文。为了帮助您，我需要您提供具体的Python代码或数据处理的示例，以便我可以添加中文注释并保持代码结构不变。

请提供您希望我处理的代码示例。

In [ ]:
# 现在对Level 3数据进行相同的光度测量

ref_image = lvl3  # 将Level 3数据文件赋值给ref_image

ref_fits = fits.open(ref_image)  # 打开FITS文件以读取数据

ref_data = fits.open(ref_image)['SCI', 1].data  # 从FITS文件中提取科学数据部分

# 使用简单归一化方法对数据进行归一化处理
norm1 = simple_norm(ref_data, stretch='linear', min_cut=-1, max_cut=10)

# 使用imshow函数显示图像，设置原点在下方，应用归一化和灰度色图
plt.imshow(ref_data, origin='lower', norm=norm1, cmap='gray')

# 隐藏坐标轴的刻度和标签
plt.gca().tick_params(labelcolor='none', axis='both', color='none')

plt.show()  # 显示图像

In [ ]:
# 从WebbPSF获取点扩散函数（PSF），并将其滴灌到源位置

# 开发说明：需要为Level3数据添加网格能力

jwst3_obs = space_phot.observation3(lvl3)  # 调用space_phot模块中的observation3函数，处理Level3数据

In [ ]:
# 现在循环遍历所有星体并构建光度表

counter = 0.  # 初始化计数器

badindex = []  # 初始化坏索引列表

for source_location in skycoords:  # 遍历每个星体的位置

    tic = time.perf_counter()  # 记录开始时间

    print('Starting', counter + 1., ' of', len(skycoords), ':', source_location)  # 打印当前处理的星体信息

    psf3 = space_phot.get_jwst3_psf(jwst_obs, jwst3_obs, source_location, num_psfs=4)  # 获取点扩散函数（PSF）

    jwst3_obs.psf_photometry(  # 进行PSF光度测量

        psf3,  # 传入PSF数据

        source_location,  # 传入星体位置

        bounds={  # 设置参数边界

            'flux': [-1000, 10000],  # 流量边界

            'centroid': [-2, 2],  # 重心边界

            'bkg': [0, 50]  # 背景边界

        },

        fit_width=5,  # 拟合宽度

        fit_bkg=True,  # 是否拟合背景

        fit_flux=True  # 是否拟合流量

    )

    jwst3_obs.plot_psf_fit()  # 绘制PSF拟合图

    plt.show()  # 显示图形

    ra = jwst3_obs.psf_result.phot_cal_table['ra'][0]  # 获取RA坐标

    dec = jwst3_obs.psf_result.phot_cal_table['dec'][0]  # 获取DEC坐标

    mag_lvl3psf = jwst3_obs.psf_result.phot_cal_table['mag'][0]  # 获取光度

    magerr_lvl3psf = jwst3_obs.psf_result.phot_cal_table['magerr'][0]  # 获取光度误差

    if counter == 0:  # 如果是第一个星体

        df = pd.DataFrame(np.array([[ra, dec, mag_lvl3psf, magerr_lvl3psf]]), columns=['ra', 'dec', 'mag', 'magerr'])  # 创建数据框

    else:  # 如果不是第一个星体

        df = pd.concat([df, pd.DataFrame(np.array([[ra, dec, mag_lvl3psf, magerr_lvl3psf]]))], ignore_index=True)  # 追加数据到数据框

    counter = counter + 1.  # 更新计数器

    toc = time.perf_counter()  # 记录结束时间

    print("Elapsed Time for Photometry:", toc - tic)  # 打印光度测量的耗时

<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="空间望远镜标志" width="200px"/>